In [2]:
from bs4 import BeautifulSoup as bs
import requests
url = 'https://fmdataba.com/19/p/1165/lionel-messi/'

In [80]:
import pandas as pd

r = requests.get('https://fmdataba.com/19/p/1165/lionel-messi/', headers = {'User-Agent':'Mozilla/5.0'})
soup = bs(r.content, 'html.parser')
abilities = ['TECHNICAL', 'MENTAL' , 'PHYSICAL']

def get_abilities(soup, keyword):
    table = soup.select_one('div:has(h3:contains("' + ability + '")) ~ div > table')
    d = {item.select_one('td:nth-child(odd)').text:item.select_one('td:nth-child(even)').text for item in table.select('tr')}
    return d

results = {}

for ability in abilities:
    results[ability] = get_abilities(soup, ability)
    
# Get Teammates URLs

table = soup.select_one('div:has(h3:contains("' + 'Teammates' + '")) ~ div > table')
# print('table', table.find_all('a')['href'])
for a in table.find_all('a', href=True):
    print("Found the URL:", a['href'])

Found the URL: https://fmdataba.com/19/p/1170/luis-suarez/
Found the URL: https://fmdataba.com/19/p/1154/philippe-coutinho/
Found the URL: https://fmdataba.com/19/p/1156/ousmane-dembele/
Found the URL: https://fmdataba.com/19/p/1134/sergio-busquets/
Found the URL: https://fmdataba.com/19/p/1112/gerard-pique/
Found the URL: https://fmdataba.com/19/p/1140/ivan-rakitic/
Found the URL: https://fmdataba.com/19/p/1124/sergi-roberto/


In [106]:
url2 = 'https://fmdataba.com/19/l/4/premier-league/best-players/'

r = requests.get(url2, headers = {'User-Agent':'Mozilla/5.0'})
soup = bs(r.content, 'html.parser')
rows = soup.find("table").find("tbody").find_all("tr")
# rows[0].find_all('a', href=True)

for i, a in enumerate(rows):
    if a.find_all('a', href=True):
        print(i, a.find_all('a', href=True)[1]['href'])

0 https://fmdataba.com/19/p/3771/eden-hazard/
1 https://fmdataba.com/19/p/4262/kevin-de-bruyne/
2 https://fmdataba.com/19/p/4599/harry-kane/
3 https://fmdataba.com/19/p/4190/mohamed-salah/
5 https://fmdataba.com/19/p/4278/sergio-aguero/
6 https://fmdataba.com/19/p/4588/christian-eriksen/
7 https://fmdataba.com/19/p/4189/roberto-firmino/
8 https://fmdataba.com/19/p/4270/david-silva/
9 https://fmdataba.com/19/p/4348/paul-pogba/
10 https://fmdataba.com/19/p/3364/pierre-emerick-aubameyang/
11 https://fmdataba.com/19/p/3744/n-golo-kante/
12 https://fmdataba.com/19/p/4269/leroy-sane/
13 https://fmdataba.com/19/p/4271/raheem-sterling/
14 https://fmdataba.com/19/p/4127/virgil-van-dijk/
15 https://fmdataba.com/19/p/4187/sadio-mane/
16 https://fmdataba.com/19/p/4586/dele-alli/
17 https://fmdataba.com/19/p/4292/david-de-gea/
18 https://fmdataba.com/19/p/4267/riyad-mahrez/
19 https://fmdataba.com/19/p/4211/vincent-kompany/
21 https://fmdataba.com/19/p/3357/mesut-ozil/
22 https://fmdataba.com/19/p/

In [3]:
# parse player urls on a single league

# url = 'https://fmdataba.com/19/l/4/premier-league/best-players/'

urls = ['https://fmdataba.com/19/l/4/premier-league/best-players/',
        'https://fmdataba.com/19/l/11/championship/best-players/',
       'https://fmdataba.com/19/l/12/league-one/best-players/',
       'https://fmdataba.com/19/l/13/league-two/best-players/',
        'https://fmdataba.com/19/l/28/national-league/best-players/',
       'https://fmdataba.com/19/l/1/la-liga/best-players/',
       'https://fmdataba.com/19/l/15/segunda-division/best-players/',
       'https://fmdataba.com/19/l/17/serie-a/best-players/',
       'https://fmdataba.com/19/l/31/serie-b/best-players/',
       'https://fmdataba.com/19/l/9/ligue-1/best-players/',
       'https://fmdataba.com/19/l/22/ligue-2/best-players/',
       'https://fmdataba.com/19/l/6/bundesliga/best-players/',
       'https://fmdataba.com/19/l/20/2-bundesliga/best-players/']

with open('fm_urls.txt', 'w') as f:
    for url in urls:
        for i in range(12):
            url2 = url + str(i)
            r = requests.get(url2, headers = {'User-Agent':'Mozilla/5.0'})
            soup = bs(r.content, 'html.parser')
            rows = soup.find("table").find("tbody").find_all("tr")
            for r in rows:
                if r.find_all('a', href=True):
                    f.write(r.find_all('a', href=True)[1]['href'] + '\n')